In [ ]:
#default_exp experiments

# Experiments

> refactor modelling, experiment functionality

In [ ]:
#export
from torchtools.core import *
from torchtools.data import *
from torchtools.models import *
from torchtools.datasets import *
from torchtools.augmentations import *
from torchtools.datablock import *
from torchtools.dataloader import *

In [ ]:
#export
import pandas as pd
import numpy as np
from fastai2.basics import *

### Worklflow with Discrete Variables

### Data Setup

In [ ]:
#export
## data config
ds_fn = 'bi_sample_anon.csv'
ds_path = Path('./data/custom')

trn_end = 120000
val_end = 160000
test_end = 200000
splits = (L(range(trn_end)), L(range(trn_end, val_end)))
ds_config = f'{int(trn_end/1000)}_{int(val_end/1000)}_{int(test_end/1000)}'

col_config = '6chan_anon_discrete'
cols_c, cols_d, cols_y, n_train = get_discrete_config()

ds_source = Path(ds_fn).stem
ds_full_path = Path(Path(ds_path)/ds_fn)

dataset_name = f'{ds_source}_{col_config}_{cols_y}_{ds_config}'



data_params = defaultdict(lambda:None, {'ds_fn':ds_fn, 'ds_path':ds_path, 'trn_end':trn_end, 'val_end':val_end, 'splits':splits,
              'col_config':col_config, 'cols_c':cols_c, 'cols_d':cols_d, 'cols_y':cols_y, 
               'ds_full_path':ds_full_path, 'dataset_name':dataset_name})



In [ ]:
nrows=200000
df_main = pd.read_csv(data_params['ds_full_path'], nrows=nrows)

# cols_c, cols_d, cols_y, n_train = get_discrete_config()
# splits = TSSplitter()(df_main)

In [ ]:
df_small = df_main.iloc[:10000]
splits_small = TSSplitter()(df_small)

In [ ]:
splits_small

((#8000) [0,1,2,3,4,5,6,7,8,9...],
 (#2000) [8000,8001,8002,8003,8004,8005,8006,8007,8008,8009...])

In [ ]:
#export
#tsai.data.core
## slightly adapted version
##NOTE TODO: Why does _ytype=TensorFloat not work (autograd fails)
class TSDatasets3(NumpyDatasets):
    "A dataset that creates tuples from X (and y) and applies `item_tfms`"
    _xtype, _xdistype, _ytype = TSTensor, TSIntTensor, None # Expected X and y output types (torch.Tensor - default - or subclass)
    def __init__(self, X=None, X_dis=None, y=None, items=None, sel_vars=None, sel_steps=None, tfms=None, tls=None, n_inp=None, dl_type=None,
                 inplace=True, **kwargs):
        self.inplace = inplace
        if tls is None:
            X = itemify(to3darray(X), tup_id=0)
            X_dis = itemify(to3darray(X_dis), tup_id=0) if X_dis is not None else X_dis
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X,)) if y is None else tuple((X,y))
            if X_dis is not None: items = tuple((X, X_dis, y)) if y is not None else tuple(X, X_dis,)
            self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
            
#         if X_dis is not None: self.X_dis = X_dis
       
        self.sel_vars = ifnone(sel_vars, slice(None))
        self.sel_steps = ifnone(sel_steps,slice(None))
#         self.splits_help = splits
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        if len(self.tls[0]) > 0:
            _tls_types = [self._xtype, self._ytype] if len(self.tls)==2 else [self._xtype, self._xdistype, self._ytype]
#             print(_tls_types)
#             print(len(self.tls))
#             for tl,_typ in zip(self.tls, _tls_types):
#                 print (len(tl), _typ, type(tl[0]), isinstance(tl[0], torch.Tensor))
            self.types = L([ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) for 
                            tl,_typ in zip(self.tls, _tls_types)])
    
            self.types = L([ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.nn.Sequential) else tensor) for 
                            tl,_typ in zip(self.tls, _tls_types)])
            if self.inplace and X and y: self.ptls=L(
                [tensor(X), tensor(y)]) if not X_dis else L([tensor(X), tensor(X_dis), tensor(y)])
            else:
                self.ptls = L([tl if not self.inplace else tl[:] if type(tl[0]).__name__ == 'memmap' else 
                               tensor(stack(tl[:])) for tl in self.tls])

    def __getitem__(self, it):
        
#         for i,(ptl,typ) in enumerate(zip(self.ptls,self.types)):
#             print (i, typ)
        
#         return tuple([typ(ptl[it])[...,self.sel_vars, self.sel_steps] if i==0 else 
#                       typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
        ## do not enable slicing for now 
        return tuple([typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
    

    def subset(self, i): 
        if self.inplace:
            X = self.ptls[0][self.splits[i]]
            y = self.ptls[-1][self.splits[i]]
            X_dis = None if len(self.ptls)==2 else self.ptls[1][self.splits[i]]
            print(X.shape, y.shape, X_dis.shape)
            return type(self)(X=X, X_dis=X_dis, y=y, n_inp=self.n_inp, 
                                           inplace=self.inplace, tfms=self.tfms,
                                           sel_vars=self.sel_vars, sel_steps=self.sel_steps)
        
        
        else:
            return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, 
                                           inplace=self.inplace, tfms=self.tfms,
                                           sel_vars=self.sel_vars, sel_steps=self.sel_steps)
    @property
    def vars(self): return self[0][0].shape[-2]
    @property
    def len(self): return self[0][0].shape[-1]


In [ ]:
items, _ = df_to_items_discrete(df_small, [cols_c, cols_d], cols_y, n_train)
Xc,Xd,y = items_to_arrays(items)

(10000, 4, 10) (10000, 2, 10)


In [ ]:
splits_test = L(L(range(7000)), L(range(7000,9000)), L(range(9000,10000)))

In [ ]:
splits_test

(#3) [(#7000) [0,1,2,3,4,5,6,7,8,9...],(#2000) [7000,7001,7002,7003,7004,7005,7006,7007,7008,7009...],(#1000) [9000,9001,9002,9003,9004,9005,9006,9007,9008,9009...]]

In [ ]:
dsets = TSDatasets3(X=Xc, X_dis=Xd, y=y, splits=splits_test)

In [ ]:
ds = [dsets.subset(i) for i in range(dsets.n_subsets)]

torch.Size([7000, 4, 10]) torch.Size([7000]) torch.Size([7000, 2, 10])
torch.Size([2000, 4, 10]) torch.Size([2000]) torch.Size([2000, 2, 10])
torch.Size([1000, 4, 10]) torch.Size([1000]) torch.Size([1000, 2, 10])


In [ ]:
dls = TSDataLoaders.from_dsets(*ds, batch_tfms=[TSStandardize()])

In [ ]:
dls.n_subsets

2

In [ ]:
dls.dataset.n_subsets

2

In [ ]:
dsets.n_subsets

3

In [ ]:
dls[2].n_subsets

2

In [ ]:
dls[2]

In [ ]:
TSDataLoaders??

In [ ]:
DataLoaders??

### Dataloaders Setup

In [ ]:
#export
def get_dls(df, cols_c, cols_y, splits, cols_d=None, bs=[256, 512], ds_type=TSDatasets3):
    '''
    create dataloaders
    '''
    if cols_d is not None:
        items, _ = df_to_items_discrete(df, [cols_c, cols_d], cols_y, n_train)
    else:
        items, _ = df_to_items(df, cols_c, cols_y, n_train)
        
    if cols_d: Xc,Xd,y = items_to_arrays(items)
    else: (Xc,y), Xd = items_to_arrays(items), None
    print(ds_type)
    dsets = ds_type(X=Xc, X_dis=Xd, y=y, splits=splits)
    print(dsets.n_subsets)
    ss = TSStandardize()
    ds = [dsets.subset(i) for i in range(dsets.n_subsets)]
    dls = TSDataLoaders.from_dsets(*ds, bs=[128]+[256]*len(splits), batch_tfms=[ss])
#     dls = TSDataLoaders.from_dsets(dsets.train, bs=[128,128])
    
    return dls

In [ ]:
%%time
dls = get_dls(df_small, cols_c, cols_y, splits_test, cols_d=cols_d, ds_type=TSDatasets3)

(10000, 4, 10) (10000, 2, 10)
<class '__main__.TSDatasets3'>
3
torch.Size([7000, 4, 10]) torch.Size([7000]) torch.Size([7000, 2, 10])
torch.Size([2000, 4, 10]) torch.Size([2000]) torch.Size([2000, 2, 10])
torch.Size([1000, 4, 10]) torch.Size([1000]) torch.Size([1000, 2, 10])
CPU times: user 1.08 s, sys: 64.8 ms, total: 1.15 s
Wall time: 609 ms


In [ ]:
dls.n_subsets

2

In [ ]:
dls[2]

In [ ]:
dls.one_batch()

(TSTensor([[[-0.3181,  1.3520, -0.2926,  ...,  1.3520, -0.2613,  1.3520],
          [-1.5024,  1.3520, -1.0529,  ...,  1.3520,  1.3520,  0.0159],
          [-0.2714,  1.3520, -0.1850,  ...,  1.3520, -0.3939,  1.3520],
          [-1.4054,  1.3520, -1.4363,  ...,  1.3520, -1.6336,  1.3520]],
 
         [[-1.3615, -0.3423,  1.3520,  ...,  1.3520, -0.3521, -0.5039],
          [-1.4054,  0.0159,  1.3520,  ...,  1.3520, -1.3202, -1.4054],
          [-0.8930,  1.3520, -0.6799,  ..., -2.8886, -0.1749, -0.8297],
          [-1.2688,  1.3520,  0.0159,  ..., -1.2445,  1.3520, -1.5938]],
 
         [[-0.5316,  1.3520, -0.2823,  ..., -0.3140, -0.1644,  1.3520],
          [ 0.0159,  0.0159,  0.0159,  ...,  0.0159,  0.0159,  0.0159],
          [-0.4864, -1.0040,  1.3520,  ..., -1.0198, -0.7305, -1.7655],
          [ 0.0159,  0.0159,  0.0159,  ...,  0.0159,  0.0159,  0.0159]],
 
         ...,
 
         [[-1.0883,  1.3520, -0.7347,  ...,  1.3520, -0.7519,  1.3520],
          [-0.7182,  1.3520,  0.0159,

In [ ]:
%%time
dls = get_dls(df_main, cols_c, cols_y, splits, cols_d=cols_d, ds_type=TSDatasets3)

(200000, 4, 10) (200000, 2, 10)
CPU times: user 13.1 s, sys: 1.1 s, total: 14.2 s
Wall time: 13.8 s


In [ ]:
%%time
#TSDatasets2
dls = get_dls(df_main, cols_c, cols_y, splits, cols_d=cols_d, ds_type=TSDatasets2)

(200000, 4, 10) (200000, 2, 10)
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, None]
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, None]
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, None]
CPU times: user 1min 3s, sys: 508 ms, total: 1min 4s
Wall time: 1min 3s


### Basic Training

In [ ]:
#export
def _remove_augs(dls):
    '''
    remove augmentation transforms from dls.after_batch
    '''
    fs = [f for f in dls.after_batch.fs if not issubclass(type(f), AugTransform)]
    print(fs)
    dls.after_batch.fs.clear()
    for f in fs: dls.after_batch.add(f)
    print(dls.after_batch, dls.train.after_batch)

In [ ]:
dls = get_dls(df_small, data_params['cols_c'], data_params['cols_y'], splits_test, 
              cols_d=data_params['cols_d'])
dls.after_batch.add(Augmix())

(10000, 4, 10) (10000, 2, 10)
<class '__main__.TSDatasets3'>
3
torch.Size([8000, 4, 10]) torch.Size([8000]) torch.Size([8000, 2, 10])
torch.Size([1000, 4, 10]) torch.Size([1000]) torch.Size([1000, 2, 10])
torch.Size([1000, 4, 10]) torch.Size([1000]) torch.Size([1000, 2, 10])
tfms None


In [ ]:
for dl in dls:
    print(dl)

In [ ]:
print(dls.after_batch, dls.train.after_batch, dls.valid.after_batch)
_remove_augs(dls)
print(dls.after_batch, dls.train.after_batch, dls.valid.after_batch)

Pipeline: TSStandardize -> Augmix Pipeline: TSStandardize -> Augmix Pipeline: TSStandardize
[TSStandardize: (TSTensor,object) -> encodes
(NumpyTensor,object) -> encodes ]
Pipeline: TSStandardize Pipeline: TSStandardize
Pipeline: TSStandardize Pipeline: TSStandardize Pipeline: TSStandardize


In [ ]:
#export
def run_training(dls, arch=None, seed=1234, n_epochs=None, max_lr=None, wd=None, 
                 loss_fn_name=None, alpha=None, metrics=unweighted_profit, 
                 N=2, magnitude=0.1, pct_start=0.3, div_factor=25.0, aug='randaugment', **kwargs):
    # model = ResNetSig(db.features, db.c).to(device)
    '''
    run a training cycle
    parameterization important for keeping track
    
    '''
    assert loss_fn_name and n_epochs, 'must pass loss_fn_name, and n_epochs'
    
    print(f'pct_start: {pct_start} div_factor: {div_factor}')
    set_seed(seed)
#     model = arch(db.features, db.c)
    model = arch(6,1)
    
    _remove_augs(dls)
    augs = RandAugment(N=N, magnitude=magnitude, verbose=True) if aug=='randaugment' else Augmix(
        N=N, magnitude=magnitude, verbose=True)
#     augs  = Augmix(verbose=True)
    dls.after_batch.add(augs)       
    loss_fn = get_loss_fn(loss_fn_name, alpha=alpha)
    print(loss_fn)
    
    learn = Learner(dls, model, loss_func=loss_fn, metrics=metrics)

    learn.fit_one_cycle(n_epochs, max_lr, wd=wd, pct_start=pct_start, div_factor=div_factor)
#     learn.recorder.plot_losses()
#     learn.recorder.plot_metrics()
    return learn

In [ ]:
#export
#train params
arch = InceptionTimeD
n_epochs = 5
max_lr = 1e-5
wd = 0.03
loss_fn_name = 'leaky_loss'
alpha = 0.5
metrics = [unweighted_profit]#, partial(unweighted_profit, threshold=0.2), 
           #partial(unweighted_profit, threshold=0.5)] #[weighted_profit, unweighted_profit_0, unweighted_profit_05]
N = 3
magnitude = 0.4
bs = [64*4, 64*4*2]  ## the validation batch size is not a training parameter...
# y_range = (-1, 1) # not sure yet about this one
seed = 1234
ds_name = dataset_name          # base dataset identifier, source csv, column config, split info
ds_path = str(ds_full_path)
pct_start=0.3                   #fastai default: 0.3
div_factor = 25.0               #fastai default 25.0
aug='augmix'

#default dict?
train_params = {'arch':arch, 'n_epochs':n_epochs, 'max_lr':max_lr, 'wd':wd, 'loss_fn_name':loss_fn_name, 'alpha':alpha,
               'metrics':metrics, 'N':N, 'magnitude':magnitude, 'bs':bs, 'seed':seed, 'ds_name':ds_name,
               'pct_start':pct_start, 'div_factor':div_factor, 'aug':aug}

In [ ]:
learn = run_training(dls, **train_params) 

pct_start: 0.3 div_factor: 25.0
[TSStandardize: (TSTensor,object) -> encodes
(NumpyTensor,object) -> encodes ]
Pipeline: TSStandardize Pipeline: TSStandardize
tfms None
functools.partial(<function leaky_loss at 0x7fa0150f7830>, alpha=0.5)


epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.100198,-0.123712,0.314173,00:01
1,-0.063339,-0.414314,0.098768,00:01
2,-0.128210,-0.423636,-0.081150,00:01
3,-0.180203,-0.419215,-0.059166,00:01
4,-0.193404,-0.417435,0.054366,00:01


In [ ]:
#export
def _losses_from_recorder(r, metrics=False):
    idx = slice(0,2) if not metrics else slice(2,None)
    return r.values[-1][idx]
def _minmax_values_from_recorder(r, metrics=False):
    idx = [0,1] if not metrics else list(range(2, len(r.values[0])))
    f = np.min if not metrics else np.max
    return L([f(L(r.values).itemgot(i)) for i in idx]) 

In [ ]:
#export
def get_recorder_dict(recorder):
    '''
    return a dictionary containing train and validation loss and metrics values
    '''
    metrics = recorder.metrics
#     loss_values = [recorder.losses[-1].item(), recorder.val_losses[-1].item()]
    loss_values = _losses_from_recorder(recorder)
#     loss_min_values = [np.min(recorder.losses), np.min(recorder.val_losses)] 
    loss_min_values = _minmax_values_from_recorder(recorder)
    metrics_values = _losses_from_recorder(recorder, metrics=True)
#     metrics_max_values = [np.max([m[i] for m in recorder.metrics]) for i in range(len(recorder.metrics[0]))]
    metrics_max_values = _minmax_values_from_recorder(recorder, metrics=True)
    recorder_keys = ['trn_loss', 'val_loss', 'trn_loss_min', 'val_loss_min', 
                     *[f'{m.name}_{i}_value' for i,m in enumerate(metrics)], *[f'{m.name}_{i}_max' for i,m in enumerate(metrics)]]
    return dict(zip(recorder_keys, loss_values+loss_min_values+metrics_values+metrics_max_values))

In [ ]:
rd = get_recorder_dict(learn.recorder)

In [ ]:
rd

{'trn_loss': -0.1934041678905487,
 'val_loss': -0.41743481159210205,
 'trn_loss_min': -0.1934041678905487,
 'val_loss_min': -0.42363613843917847,
 'unweighted_profit_0_value': 0.05436611920595169,
 'unweighted_profit_0_max': 0.31417301297187805}

In [ ]:
#export
def _to_flat_dict(train_params):
    flat_dict={}
    for key,value in train_params.items():
        if key=='metrics':
            for i,_ in enumerate(listify(value)):
                flat_dict[f'metric_{i}'] = value[i].__name__
        elif key=='arch': flat_dict[key] = value.__name__
        else: flat_dict[key]=value
    return flat_dict

In [ ]:
#export
def _write_results(df, fn):
    if not os.path.isfile(fn):
        df.to_csv(fn, index=False)
    else:
        print('not new')
        df_old = pd.read_csv(fn)
        df_new = pd.concat([df_old, df], ignore_index=True, sort=False)
#         df.to_csv(fn, index=False, mode='a', header=False)
        df_new.to_csv(fn, index=False)

In [ ]:
train_params

{'arch': torchtools.models.InceptionTimeD,
 'n_epochs': 5,
 'max_lr': 1e-05,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metrics': [<function torchtools.core.unweighted_profit(preds, y_true, threshold=0)>],
 'N': 3,
 'magnitude': 0.4,
 'bs': [256, 512],
 'seed': 1234,
 'ds_name': 'bi_sample_anon_6chan_anon_discrete_y0_120_160_200',
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix'}

In [ ]:
_to_flat_dict(train_params)

{'arch': 'InceptionTimeD',
 'n_epochs': 5,
 'max_lr': 1e-05,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metric_0': 'unweighted_profit',
 'N': 3,
 'magnitude': 0.4,
 'bs': [256, 512],
 'seed': 1234,
 'ds_name': 'bi_sample_anon_6chan_anon_discrete_y0_120_160_200',
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix'}

In [ ]:
def run_experiment(dls, train_params, df_fn=None):
        '''
        could wrap the dataset parameters
        '''
        assert df_fn is not None, 'please specify results csv filename'
  
     
        learn = run_training(dls, **train_params)

        rec_dict = get_recorder_dict(learn.recorder)
        df_dict = dict()
        #     param_values = [arch.__name__, n_epochs, bs, seed, max_lr, wd, loss_fn_name, alpha, *[m.__name__ for m in metrics], N, magnitude]
        #     param_key = ['architecture', 'n_epochs', 'bs', 'seed', 'max_lr', 'wd', 'loss_fn', 'alpha', *[f'metric_{i}' for i,_ in enumerate(metrics)], 'N', 'magnitude']
        #     df_dict.update(train_params)
        df_dict.update(_to_flat_dict(train_params))
        df_dict.update(rec_dict)
        df_dict['Timestamp'] = str(datetime.now())
        ## store prediction in a separate file as tensors, but add filename
#         self._save_preds(test=True)
#         if self.save_model: self._save_model()
        _write_results(pd.DataFrame([df_dict], index=[0]), df_fn)
        return df_dict
    

In [ ]:
df_fn_test = Path('~/coding/python/betting/experiments/test_results.csv').expanduser()

In [ ]:
results_df = pd.read_csv(df_fn_test)

In [ ]:
results_df

,arch,n_epochs,max_lr,wd,loss_fn_name,alpha,metric_0,N,magnitude,bs,...,pct_start,div_factor,aug,trn_loss,val_loss,trn_loss_min,val_loss_min,unweighted_profit_0_value,unweighted_profit_0_max,Timestamp
0,InceptionTimeD,2,0.00001,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.128333,-0.711629,-0.133325,-0.738477,0.000000,1.912173,2020-05-21 14:36:42.977545
1,InceptionTimeD,5,0.00001,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.181375,-0.683619,-0.181375,-0.740593,0.000000,1.912173,2020-05-21 14:38:17.612064
2,InceptionTimeD,3,0.00003,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.181424,-0.655837,-0.181424,-0.730417,0.000000,1.912173,2020-05-21 16:19:49.366239
3,InceptionTimeD,3,0.00010,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.204159,-0.671866,-0.204159,-0.711356,0.000000,0.000000,2020-05-21 16:20:01.197565
4,InceptionTimeD,3,0.00030,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.282855,-0.568804,-0.282855,-0.694458,0.044329,0.044329,2020-05-21 16:20:12.393597
5,InceptionTimeD,4,0.00003,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.222047,-0.438311,-0.222047,-0.443605,-0.107899,0.143756,2020-05-22 08:48:22.384761
6,InceptionTimeD,4,0.00010,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.327031,-0.427454,-0.327031,-0.453769,0.055171,0.270344,2020-05-22 08:48:28.630306
7,InceptionTimeD,4,0.00030,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",...,0.3,25.0,augmix,-0.203794,-0.443539,-0.203794,-0.443539,-0.045778,0.040683,2020-05-22 08:48:34.883116


In [ ]:
train_params

{'arch': torchtools.models.InceptionTimeD,
 'n_epochs': 5,
 'max_lr': 1e-05,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metrics': [<function torchtools.core.unweighted_profit(preds, y_true, threshold=0)>],
 'N': 3,
 'magnitude': 0.4,
 'bs': [256, 512],
 'seed': 1234,
 'ds_name': 'bi_sample_anon_6chan_anon_discrete_y0_120_160_200',
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix'}

### Wrapper

In [ ]:
#export
def _dict_product(params):
            values = list(itertools.product(*params.values()))
            return [dict(zip(params.keys(), values[i])) for i in range(len(values))]

In [ ]:
data_params

defaultdict(<function __main__.<lambda>()>,
            {'ds_fn': 'bi_sample_anon.csv',
             'ds_path': Path('data/custom'),
             'trn_end': 120000,
             'val_end': 160000,
             'splits': ((#8000) [0,1,2,3,4,5,6,7,8,9...],
              (#2000) [8000,8001,8002,8003,8004,8005,8006,8007,8008,8009...]),
             'col_config': '6chan_anon_discrete',
             'cols_c': [['x0_0',
               'x0_1',
               'x0_2',
               'x0_3',
               'x0_4',
               'x0_5',
               'x0_6',
               'x0_7',
               'x0_8',
               'x0_9'],
              ['x1_0',
               'x1_1',
               'x1_2',
               'x1_3',
               'x1_4',
               'x1_5',
               'x1_6',
               'x1_7',
               'x1_8',
               'x1_9'],
              ['x3_0',
               'x3_1',
               'x3_2',
               'x3_3',
               'x3_4',
               'x3_5',
      

In [ ]:
#export
def _get_preds_fn(prefix='val'):
    return f'{prefix}_preds_{abs(hash(datetime.utcnow()))}.pt'

def _get_model_fn(prefix='model'):
    return f'{prefix}_{abs(hash(datetime.utcnow()))}' 

In [ ]:
#export
class TSExperiments:
    '''
    Wrapper class for TS modelling experiments
    '''
    def __init__(self, train_params, save_model=False, preds_path=None, model_path=None, results_path=None):
        
        self.train_params = train_params #training params can change, e.g. when running grid search
        self.save_model = save_model## models are big
        self.preds_path = ifnone(preds_path, './experiments/preds')
        self.model_path = ifnone(model_path, './experiments/models')
        self.results_path = ifnone(results_path, './experiments/results')
        
    def setup_data(self, df_base, data_params):
        cols_c, cols_d, cols_y, splits = map(data_params.get, ['cols_c', 'cols_d', 'cols_y', 'splits'])
        
        self.bs = self.train_params['bs']
        self.splits = splits
        self.dls = get_dls(df_base, cols_c, cols_y, splits, cols_d=cols_d, bs=self.bs)    
 
    def _save_preds(self, test=False):
    #         val_preds_fn = _get_preds_fn()
        preds_fn = _get_preds_fn()
        preds, y_true = self.learn.get_preds(1)
        torch.save(preds, Path(self.preds_path)/preds_fn)
        self.df_dict.update({'val_preds':preds_fn})
        if len(list(self.dls))==3:
            preds_fn = _get_preds_fn('test')
            preds, y_true = self.learn.get_preds(2)
            torch.save(preds, Path(self.preds_path)/preds_fn)
            self.df_dict.update({'test_preds':preds_fn})
    
    def _save_model(self):
    #         val_preds_fn = _get_preds_fn()
        model_fn = _get_model_fn()
        self.learn.save(model_fn)
        self.df_dict.update({'model_fn':f'{self.learn.model_dir}/{model_fn}.pth'})

    
    
    def run_training(self, arch=None, seed=1234, n_epochs=None, max_lr=None, wd=None, 
                     loss_fn_name=None, alpha=None, metrics=unweighted_profit, 
                     N=2, magnitude=0.1, pct_start=0.3, div_factor=25.0, aug='randaugment', **kwargs):
        # model = ResNetSig(db.features, db.c).to(device)
        '''
        run a training cycle
        parameterization important for keeping track
        '''
        assert loss_fn_name and n_epochs, 'must pass loss_fn_name, and n_epochs'

        print(f'pct_start: {pct_start} div_factor: {div_factor}')
        set_seed(seed)
    #     model = arch(db.features, db.c)
        model = arch(6,1)

        _remove_augs(self.dls)
        augs = RandAugment(N=N, magnitude=magnitude, verbose=True) if aug=='randaugment' else Augmix(
            N=N, magnitude=magnitude, verbose=True)
    #     augs  = Augmix(verbose=True)
        self.dls.after_batch.add(augs)       
        loss_fn = get_loss_fn(loss_fn_name, alpha=alpha)
        print(loss_fn)
        learn = Learner(self.dls, model, loss_func=loss_fn, metrics=metrics, model_dir=self.model_path)

        learn.fit_one_cycle(n_epochs, max_lr, wd=wd, pct_start=pct_start, div_factor=div_factor)
    #     learn.recorder.plot_losses()
    #     learn.recorder.plot_metrics()
        return learn


    def run_experiment(self, df_fn=None):
        '''
        could wrap the dataset parameters
        '''
        assert df_fn is not None, 'please specify results csv filename'
  
        self.learn = self.run_training(**self.train_params)
#         rec_dict = get_recorder_dict(self.learn.recorder)
        self.df_dict = dict()
        self.df_dict.update(_to_flat_dict(train_params))
        self.df_dict.update(get_recorder_dict(self.learn.recorder))
        self.df_dict['Timestamp'] = str(datetime.now())
        ## store prediction in a separate file as tensors, but add filename
        self._save_preds(test=False)
        if self.save_model: self._save_model()
        _write_results(pd.DataFrame([self.df_dict], index=[0]), Path(self.results_path)/df_fn)
#         return df_dict
    
    
   
    def run_grid_search(self, hypers:dict, df_results_fn=None):
        '''
        run hyper parameter grid search, note that this changes self.train_params
        '''
        hyper_configs = _dict_product(hypers) #list of dictionaries
        if hasattr(self, 'hyper_configs'): self.hyper_configs+=hyper_configs
        else: self.hyper_configs = hyper_configs
        for config in hyper_configs:
            self.train_params.update(config)
            print(self.train_params)
            self.run_experiment(df_fn=df_results_fn)

In [ ]:
ts_experiment = TSExperiments(train_params)

In [ ]:
data_params['splits'] = splits_small

In [ ]:
ts_experiment.setup_data(df_small, data_params)

(10000, 4, 10) (10000, 2, 10)


In [ ]:
hypers = {'max_lr':[3e-5, 1e-4, 3e-4], 'n_epochs':[7], 'N':[3], 'magnitude':[0.4]}

In [ ]:
train_params

{'arch': torchtools.models.InceptionTimeD,
 'n_epochs': 7,
 'max_lr': 0.0003,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metrics': [<function torchtools.core.unweighted_profit(preds, y_true, threshold=0)>],
 'N': 3,
 'magnitude': 0.4,
 'bs': [256, 512],
 'seed': 1234,
 'ds_name': 'bi_sample_anon_6chan_anon_discrete_y0_120_160_200',
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix'}

In [ ]:
ts_experiment.run_grid_search(hypers, df_fn_test)

{'arch': <class 'torchtools.models.InceptionTimeD'>, 'n_epochs': 7, 'max_lr': 3e-05, 'wd': 0.03, 'loss_fn_name': 'leaky_loss', 'alpha': 0.5, 'metrics': [<function unweighted_profit at 0x7fa0150f7d40>], 'N': 3, 'magnitude': 0.4, 'bs': [256, 512], 'seed': 1234, 'ds_name': 'bi_sample_anon_6chan_anon_discrete_y0_120_160_200', 'pct_start': 0.3, 'div_factor': 25.0, 'aug': 'augmix'}
pct_start: 0.3 div_factor: 25.0
[TSStandardize: (TSTensor,object) -> encodes
(NumpyTensor,object) -> encodes ]
Pipeline: TSStandardize Pipeline: TSStandardize
tfms None
functools.partial(<function leaky_loss at 0x7fa0150f7830>, alpha=0.5)


epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.061447,-0.253833,0.193756,00:01
1,-0.138268,-0.512078,-0.034383,00:01
2,-0.173151,-0.464161,-0.178399,00:01
3,-0.204135,-0.478039,-0.178399,00:01
4,-0.268758,-0.478676,-0.157899,00:01
5,-0.351980,-0.457422,-0.157899,00:01
6,-0.336728,-0.456215,-0.157899,00:01


not new
{'arch': <class 'torchtools.models.InceptionTimeD'>, 'n_epochs': 7, 'max_lr': 0.0001, 'wd': 0.03, 'loss_fn_name': 'leaky_loss', 'alpha': 0.5, 'metrics': [<function unweighted_profit at 0x7fa0150f7d40>], 'N': 3, 'magnitude': 0.4, 'bs': [256, 512], 'seed': 1234, 'ds_name': 'bi_sample_anon_6chan_anon_discrete_y0_120_160_200', 'pct_start': 0.3, 'div_factor': 25.0, 'aug': 'augmix'}
pct_start: 0.3 div_factor: 25.0
[TSStandardize: (TSTensor,object) -> encodes
(NumpyTensor,object) -> encodes ]
Pipeline: TSStandardize Pipeline: TSStandardize
tfms None
functools.partial(<function leaky_loss at 0x7fa0150f7830>, alpha=0.5)


epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.114760,-0.213118,0.253433,00:01
1,-0.197599,-0.503396,-0.177498,00:01
2,-0.293514,-0.422551,-0.006572,00:01
3,-0.347330,-0.424305,0.004674,00:01
4,-0.381413,-0.524226,-0.103994,00:01
5,-0.458140,-0.443198,0.119664,00:01
6,-0.487051,-0.410117,0.120056,00:01


not new
{'arch': <class 'torchtools.models.InceptionTimeD'>, 'n_epochs': 7, 'max_lr': 0.0003, 'wd': 0.03, 'loss_fn_name': 'leaky_loss', 'alpha': 0.5, 'metrics': [<function unweighted_profit at 0x7fa0150f7d40>], 'N': 3, 'magnitude': 0.4, 'bs': [256, 512], 'seed': 1234, 'ds_name': 'bi_sample_anon_6chan_anon_discrete_y0_120_160_200', 'pct_start': 0.3, 'div_factor': 25.0, 'aug': 'augmix'}
pct_start: 0.3 div_factor: 25.0
[TSStandardize: (TSTensor,object) -> encodes
(NumpyTensor,object) -> encodes ]
Pipeline: TSStandardize Pipeline: TSStandardize
tfms None
functools.partial(<function leaky_loss at 0x7fa0150f7830>, alpha=0.5)


epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.100034,-0.314245,0.193756,00:01
1,-0.264312,-0.551215,-0.416235,00:01
2,-0.355112,-0.438786,0.075726,00:01
3,-0.339290,-0.234057,0.053099,00:01
4,-0.427105,-0.291003,0.130943,00:01
5,-0.521998,-0.219214,0.025726,00:01
6,-0.614366,-0.186999,0.130943,00:01


not new


In [ ]:
results_df = pd.read_csv(df_fn_test)

In [ ]:
pd.options.display.max_columns = 99
results_df

,arch,n_epochs,max_lr,wd,loss_fn_name,alpha,metric_0,N,magnitude,bs,seed,ds_name,pct_start,div_factor,aug,trn_loss,val_loss,trn_loss_min,val_loss_min,unweighted_profit_0_value,unweighted_profit_0_max,Timestamp
0,InceptionTimeD,2,0.00001,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.128333,-0.711629,-0.133325,-0.738477,0.000000,1.912173,2020-05-21 14:36:42.977545
1,InceptionTimeD,5,0.00001,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.181375,-0.683619,-0.181375,-0.740593,0.000000,1.912173,2020-05-21 14:38:17.612064
2,InceptionTimeD,3,0.00003,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.181424,-0.655837,-0.181424,-0.730417,0.000000,1.912173,2020-05-21 16:19:49.366239
3,InceptionTimeD,3,0.00010,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.204159,-0.671866,-0.204159,-0.711356,0.000000,0.000000,2020-05-21 16:20:01.197565
4,InceptionTimeD,3,0.00030,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.282855,-0.568804,-0.282855,-0.694458,0.044329,0.044329,2020-05-21 16:20:12.393597
5,InceptionTimeD,4,0.00003,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.222047,-0.438311,-0.222047,-0.443605,-0.107899,0.143756,2020-05-22 08:48:22.384761
6,InceptionTimeD,4,0.00010,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.327031,-0.427454,-0.327031,-0.453769,0.055171,0.270344,2020-05-22 08:48:28.630306
7,InceptionTimeD,4,0.00030,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.203794,-0.443539,-0.203794,-0.443539,-0.045778,0.040683,2020-05-22 08:48:34.883116
8,InceptionTimeD,7,0.00003,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_anon_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.336728,-0.456215,-0.351980,-0.512078,-0.157899,0.193756,2020-05-22 09:45:35.273860
9,InceptionTimeD,7,0.00010,0.03,leaky_loss,0.5,unweighted_profit,3,0.4,"[256, 512]",1234,bi_sample_anon_6chan_anon_discrete_y0_120_160_200,0.3,25.0,augmix,-0.487051,-0.410117,-0.487051,-0.524226,0.120056,0.253433,2020-05-22 09:45:46.067508
